In [ ]:
import sys
sys.path.append("../")
import core.optimizers as O
optimizer=O.Adam()
from core.MLp_layer import Layer
import torch

from GAN import Generator,Discriminator,Generator_loss,Discriminator_loss


generator=Generator()
discriminator=Discriminator()
generator.add_layer(2,2,"Xaviernormal")
discriminator.add_layer(2,1,"Xaviernormal")
gen_optimizer=O.Adam()
desc_optimizer=O.Adam()
gloss=Generator_loss()
dloss=Discriminator_loss()



def backward_propagation(optimizer,weights, lr, t):

        for w in weights:
            w.data = optimizer.update(w, lr, t)

        for w in weights:
            w.grad.zero_()

def minibatch_SGD_train(epochs,noise,data,batch_size):
        """
        Mini-Batch Stochastic Gradient Descent training.
        """
        num_batches = len(data) // batch_size
        

        for epoch in range(epochs):
            indices = torch.randperm(len(data))
            data, noise = data[indices], noise[indices]
            Descepoch_loss=0.0
            Genepoch_loss=0.0

            for i in range(num_batches):
                start, end = i * batch_size, (i + 1) * batch_size
                data_batch= data[start:end]
                noise_batch=noise[start:end]
                
                # Train Discriminator
                gen_x=generator.forward(noise_batch)
                gen_y=discriminator.forward(gen_x.detach())
                y=discriminator.forward(data_batch)

                Descloss=dloss.compute(y,gen_y)
                Descloss.backward()

                with  torch.no_grad():
                     backward_propagation(desc_optimizer,discriminator.weights,0.01,epoch+1)
                
                # Train Generator
                gen_x=generator.forward(noise_batch)
                gen_y=discriminator.forward(gen_x)
                Genloss =gloss.compute(gen_y)
                Genloss.backward()     
                with  torch.no_grad():
                     backward_propagation(gen_optimizer,generator.weights,0.01,epoch+1)
                

                     
                Descepoch_loss += Descloss.item()
                Genepoch_loss+=Genloss.item()
                

            Descepoch_loss /= num_batches
            Genepoch_loss/=num_batches


            print(f"{epoch+1} | Gen Loss: {Genepoch_loss:.4f} | Desc Loss: {Descepoch_loss:.4f}")

In [62]:
def get_real_data(n_samples=1000):
    """Retourne des données réelles suivant une distribution Gaussienne 2D"""
    X = torch.normal(0.0, 1, (n_samples, 2))
    A = torch.tensor([[1, 2], [-0.1, 0.5]])
    b = torch.tensor([1, 2])
    real_data = torch.matmul(X, A) + b
    return real_data

# 2. Fonction pour le BRUIT (x_noise)
def get_noise(batch_size, noise_dim=2):
    """Retourne du bruit aléatoire pour le générateur"""
    noise = torch.randn(batch_size, noise_dim)
    return noise

x=get_real_data(100)
noise=get_noise(100)
minibatch_SGD_train(10,noise,x,50)

1 | Gen Loss: 0.7499 | Desc Loss: 1.1856
2 | Gen Loss: 0.7380 | Desc Loss: 1.1669
3 | Gen Loss: 0.7276 | Desc Loss: 1.1538
4 | Gen Loss: 0.7223 | Desc Loss: 1.1367
5 | Gen Loss: 0.7136 | Desc Loss: 1.1298
6 | Gen Loss: 0.7084 | Desc Loss: 1.1215
7 | Gen Loss: 0.7042 | Desc Loss: 1.1142
8 | Gen Loss: 0.7003 | Desc Loss: 1.1095
9 | Gen Loss: 0.6971 | Desc Loss: 1.1051
10 | Gen Loss: 0.6948 | Desc Loss: 1.1015
